In [2]:
import pandas as pd


def calculate_current_cgpa(student_id):
    studentScore = 0  # Initialize total score
    total_credits_passed = 0  # Initialize total credits of passed courses
    a = pd.read_csv('dataset/allstudentdata/' + student_id + '.csv')
    sems = a['Semester Offer'].unique()

    # Initialize student score and total credits passed if not already initialized
    studentScore = 0
    total_credits_passed = 0
    course_latest_info = {}
    # Loop through each semester
    passed_courses = set()
    warning_count = 0
    for sm in sems:
        sem = a[a['Semester Offer'] == sm]
        
        # Get the GPA, CreditHour, and CourseID data for the semester
        data = sem[['GPA', 'CreditHour', 'CourseID']].values.tolist()
        
        # Calculate SGPA for the semester
        score = sum([x * b for x, b, _ in data])
        credits_count = sum([b for _, b, _ in data])
        SGPA = score / credits_count

        for x, b, course_id in data:
            course_latest_info[course_id] = (x, b)
        # Calculate total credits passed (excluding failed courses)
        # cgpaSCore = 
        # credits_passed = 0
        # for x, b, course_id in data:
        #     if course_id not in passed_courses:
        #         credits_passed += b
        #         passed_courses.add(course_id)
        
        # # Update total score and total credits passed
        # studentScore += score
        # total_credits_passed += credits_passed
        
        # # Calculate CGPA
        # CGPA = studentScore / total_credits_passed 
        # loop through course_latest_info and calculate the total score and total credits passed for the student
        studentScore = 0
        total_credits_passed = 0
        for course_id, (x, b) in course_latest_info.items():
            studentScore += x * b
            total_credits_passed += b

        CGPA = studentScore / total_credits_passed
        if CGPA < 2:
            warning_count +=1
        else:
            warning_count = 0
    print(CGPA, 'cgpa')
    return CGPA


In [5]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
import math
from itertools import combinations

class Student:
    def __init__(self, student_id, level_of_understanding, preferences, warning_status, cgpa, current_semester):
        self.student_id = student_id
        self.level_of_understanding = level_of_understanding
        self.preferences = preferences
        self.warning_status = warning_status
        self.cgpa = cgpa
        self.current_semester = current_semester

class Course:
    def __init__(self, course_id, name, category, chain_courses, grade, semester_offer, credit_hours, repeat_status):
        self.course_id = course_id
        self.name = name
        self.category = category
        self.chain_courses = chain_courses
        self.grade = grade
        self.semester_offer = semester_offer
        self.credit_hours = credit_hours
        self.repeat_status = repeat_status

class Transcript:
    def __init__(self, student_id, courses_taken):
        self.student_id = student_id
        self.courses_taken = courses_taken  # List of Course objects

def load_data():
    student_df = pd.read_csv('dataset/Students.csv')
    course_df = pd.read_csv('dataset/courses.csv')
    return student_df, course_df

def categorize_students(student):
    if student.cgpa >= 1.85:
        return 'near_2.0'
    elif student.cgpa >= 1.5:
        return 'near_1.5'
    else:
        return 'less_than_1.5'

def classify_student_type(transcript):
    grades = [course.grade for course in transcript.courses_taken]
    if sum(grades) / len(grades) > 2.5:
        return 'good'
    elif sum(grades) / len(grades) > 1.5:
        return 'average'
    else:
        return 'below_average'

def get_courses_taken(student_id, transcripts):
    for transcript in transcripts:
        if transcript.student_id == student_id:
            return transcript.courses_taken
    return []

def calculate_gpa(courses_taken):
    
    total_points = 0
    total_credits = 0
    for course in courses_taken:
        total_points += course.grade * course.credit_hours
        total_credits += course.credit_hours
        print(total_points, 'total_points')
        print(total_credits, 'total_credits')
    return total_points / total_credits

def calculate_category_average_gpa(transcript):
    category_gpa = defaultdict(list)
    for course in transcript:
        if not math.isnan(course.grade):
            category_gpa[course.category].append(course.grade)
    
    category_avg_gpa = {}
    for category, grades in category_gpa.items():
        category_avg_gpa[category] = sum(grades) / len(grades)
    
    return category_avg_gpa

def find_combinations_to_clear_warning(student, available_courses, transcripts):
    taken_courses = get_courses_taken(student.student_id, transcripts)
    category_avg_gpa = calculate_category_average_gpa(taken_courses)
    print(category_avg_gpa, 'gpa')
    stid = f'k{student.student_id[0:2]}{student.student_id.split("-")[1]}'
    
    current_gpa = calculate_current_cgpa(stid)
    required_gpa = 2.0
    if current_gpa >= required_gpa:
        return []
    successful_combinations = []
    num_courses = len(available_courses)

    for r in range(1, num_courses + 1):
        for combo in combinations(available_courses, r):
            new_courses = list(combo)
            for course in new_courses:
                # Estimate GPA for new courses based on category average
                estimated_grade = category_avg_gpa.get(course.category, 0)
                course.grade = estimated_grade
            # combined_courses = list(set(taken_courses)) + new_courses
            # for combined_course in combined_courses:
            #     print(combined_course.name)
            common_courses = set([course.course_id for course in taken_courses]).intersection([course.course_id for course in new_courses])
            print([cm for cm in common_courses])
            # if common courses gpa is less than 3.0, then subtract that course from new_courses
            for common_course in common_courses:
                for course in new_courses:
                    if course.course_id == common_course and course.grade >= 3:
                        print(course.name, 'name')
                        new_courses.remove(course)

                
            if len(new_courses) == 0:
                continue
            new_gpa = calculate_gpa(new_courses)
            print(new_gpa, 'new_courses')
            if new_gpa >= required_gpa:
                successful_combinations.append((new_courses, new_gpa))

    return successful_combinations

def grade_to_score(grade):
    grade_conversion = {'A+': 4.0, 'A': 4.0, 'A-': 3.67,  'B+': 3.33, 'B': 3.0, 'B-': 2.67, 
                        'C+': 2.33, 'C': 2.0, 'C-': 1.67, 'D+': 1.33,  'D': 1.0, 'F': 0.0}
    return grade_conversion.get(grade, 0.0)

# Load data
student_df, course_df = load_data()

# Initialize students and transcripts
all_students = []
all_transcripts = []

for _, row in student_df.iterrows():
    student = Student(row['StudentID'], None, [], row['WarningStatus'], row['CGPA'], row['CurrentSemester'])
    courses_taken = []
    student_file = f'dataset/allstudentdata/k{row["StudentID"][0:2]}{row["StudentID"].split("-")[1]}.csv'
    student_data_df = pd.read_csv(student_file)
    for _, st_row in student_data_df.iterrows():
        course_info = course_df[course_df['CourseID'] == st_row['CourseID']]
        chain_courses = []
        
        if not course_info.empty:
            course_info = course_info.iloc[0]
            chain_courses = course_info['Chain'] if not pd.isnull(course_info['Chain']) else []
        course_category = course_info['Category']
        course = Course(st_row['CourseID'], st_row['CourseName'], course_category, chain_courses, st_row['GPA'], course_info['Semester Offer'], course_info['CreditHours'], st_row['Repeat Status'])
        courses_taken.append(course)
    
    transcript = Transcript(row['StudentID'], courses_taken)
    all_transcripts.append(transcript)
    all_students.append(student)

# Example usage

student = all_students[-1]

print("Student ID:", student.student_id)
print(student.student_id)
student.level_of_understanding = ['Programming', 'Social', 'Language']
student.preferences = []

available_courses = []
total_courses = len(course_df)
num_random_courses = 10 
random_indices = [16,17,18,19,20,21,22]

for index in random_indices:
    row = course_df.iloc[index]
    chain_courses = []
    
    if not row.empty:
        chain_courses = [row['Chain']] if not pd.isnull(row['Chain']) else []
    course = Course(row['CourseID'], row['CourseName'], row['Category'], chain_courses, 0, row['Semester Offer'], row['CreditHours'], 0)
    available_courses.append(course)

successful_combinations = find_combinations_to_clear_warning(student, available_courses, all_transcripts)

print("Available Courses:")
for course in available_courses:
    print(course.course_id, course.name)
print("\n")

print("Successful Combinations to Clear Warning (CGPA > 2.0):", len(successful_combinations))
for combo, new_gpa in successful_combinations:
    course_ids = [course.course_id for course in combo]
    course_names = [course.name for course in combo]
    print(f"Courses: {course_ids} - Names: {course_names} - New CGPA: {new_gpa:.2f}")



Student ID: 21k-3084
21k-3084
{'Programming': 1.6489473684210525, 'Mathematics': 0.6666666666666666, 'Language': 3.9175, 'Social': 3.89, 'Theory': 1.0} gpa
1.9358064516129034 cgpa
['AI2001']
4.9468421052631575 total_points
3 total_credits
1.6489473684210525 new_courses
['AL2001']
1.6489473684210525 total_points
1 total_credits
1.6489473684210525 new_courses
['CL2001']
1.6489473684210525 total_points
1 total_credits
1.6489473684210525 new_courses
['CS1005']
2.0 total_points
3 total_credits
0.6666666666666666 new_courses
['CS2001']
4.9468421052631575 total_points
3 total_credits
1.6489473684210525 new_courses
['MT2005']
2.0 total_points
3 total_credits
0.6666666666666666 new_courses
['SS2003']
Psychology name
['AI2001', 'AL2001']
4.9468421052631575 total_points
3 total_credits
6.59578947368421 total_points
4 total_credits
1.6489473684210525 new_courses
['AI2001', 'CL2001']
4.9468421052631575 total_points
3 total_credits
6.59578947368421 total_points
4 total_credits
1.6489473684210525 new